In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(3,  name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [3]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

print(result)
sess.close()

42


In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [6]:
w = tf.constant(3)
x = w +2
y = x + 5
z = x * 3

with tf.Session() as sess:
    # x and w will be evaluate twice
    print(y.eval())
    print(z.eval())

10
15


In [7]:
with tf.Session() as sess:
    # x and w will only be evaluate once
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


In [8]:
import numpy as np
import tarfile
from sklearn.datasets import fetch_california_housing
from sklearn.utils import Bunch
from sklearn.datasets.california_housing import MODULE_DOCS

# housing = fetch_california_housing(data_home="dataset")

def load_local_housing_data(file_path):
    cal_housing = np.loadtxt(file_path, delimiter=",")
    columns_index =  [8, 7, 2, 3, 4, 5, 6, 1, 0]
    cal_housing = cal_housing[:, columns_index]
    feature_names =  ["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup", "Latitude", "Longitude"]
    target, data = cal_housing[:, 0], cal_housing[:,1:]

    # avg rooms = total rooms / households
    data[:, 2] /= data[:, 5]
    # avg bed rooms = total bed rooms / households
    data[:, 3] /= data[:, 5]
    # avg occupancy = population / households
    data[:, 5] = data[:, 4] / data[:, 5]
    # target in units of 100,000
    target = target / 100000.0

    housing = Bunch(data=data, target=target, feature_names=feature_names, DESCR=MODULE_DOCS)
    return housing
    
housing = load_local_housing_data("dataset/cal_housing.data")


In [9]:
m, n = housing.data.shape

housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y= tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)

theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [10]:
theta_value

array([[ -3.74651413e+01],
       [  4.35734153e-01],
       [  9.33829229e-03],
       [ -1.06622010e-01],
       [  6.44106984e-01],
       [ -4.25131839e-06],
       [ -3.77322501e-03],
       [ -4.26648885e-01],
       [ -4.40514028e-01]], dtype=float32)

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)
X_s = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")

n_epochs = 1000
learning_rate = 0.01

theta_g = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta_g")
y_pred = tf.matmul(X_s, theta_g, name="predictions")
error = y - y_pred
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X_s), error)
training_op = tf.assign(theta_g, theta_g - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0 :
            print("Epoch", epoch, "MSE =", mse.eval())

        sess.run(training_op)
    
    beta_theta_g = theta_g.eval()

print(beta_theta_g)

Epoch 0 MSE = 7.56435
Epoch 100 MSE = 2064.16
Epoch 200 MSE = 5.2637e+06
Epoch 300 MSE = 1.43931e+10
Epoch 400 MSE = 3.99497e+13
Epoch 500 MSE = 1.11784e+17
Epoch 600 MSE = 3.1426e+20
Epoch 700 MSE = 8.85876e+23
Epoch 800 MSE = 2.50113e+27
Epoch 900 MSE = 7.06787e+30
[[ -2.67341137e-01]
 [  8.40201796e+15]
 [ -8.38835802e+13]
 [  4.52816255e+16]
 [  4.12571509e+16]
 [ -1.67361293e+16]
 [ -1.44524361e+15]
 [  5.52458809e+16]
 [ -5.19802540e+16]]


In [14]:
theta_g = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta_g")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
built_in_traning_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(built_in_traning_op)
        
    beta_theta_g = theta_g.eval()
print(beta_theta_g)

Epoch 0 MSE = 6.29162
Epoch 100 MSE = 4.89379
Epoch 200 MSE = 4.8567
Epoch 300 MSE = 4.84197
Epoch 400 MSE = 4.83173
Epoch 500 MSE = 4.82426
Epoch 600 MSE = 4.81881
Epoch 700 MSE = 4.81481
Epoch 800 MSE = 4.81186
Epoch 900 MSE = 4.80969
[[ 0.02608848]
 [-0.13684154]
 [ 0.11185932]
 [ 0.31244636]
 [-0.44215155]
 [ 0.5775547 ]
 [ 0.6329658 ]
 [ 0.03567529]
 [ 0.68074369]]


In [15]:
A = tf.placeholder(tf.float32, shape=(None, 3))

B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A: [[4,5,6], [7,8,9]]})

print(B_val_1)
print(B_val_2)

[[ 6.  7.  8.]]
[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [16]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [17]:
mse_summary = tf.summary.scalar("MSE", mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

X= tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m/batch_size))

In [18]:
labels = housing.target.reshape(-1, 1)
def fetch_batch(epoch, batch_index, batch_size):
    start = batch_index * batch_size
    end = (batch_index + 1) * batch_size
    X_batch =  scaled_housing_data_plus_bias[start: end]
    y_batch = labels[start: end]
    return X_batch, y_batch
    

In [20]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()
    

In [21]:
file_writer.close()

In [27]:
n_features = 9

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(.0, name="bias1")
b2 = tf.Variable(.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")

output = tf.add(relu1, relu2, name="output")

error = y - output
mse = tf.reduce_mean(tf.square(error), name="mse")
built_in_traning_op = optimizer.minimize(mse)

In [28]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if epoch % 100 == 0:
                print("Epoch", epoch, "MSE =", mse.eval())
            sess.run(built_in_traning_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()
    

InvalidArgumentError: Shape [-1,1] has negative dimensions
	 [[Node: y_2 = Placeholder[dtype=DT_FLOAT, shape=[?,1], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'y_2', defined at:
  File "c:\users\yucai\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\yucai\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\yucai\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\yucai\appdata\local\programs\python\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\yucai\appdata\local\programs\python\python36\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "c:\users\yucai\appdata\local\programs\python\python3